# Tutorial 1: Manifold Capacity Theory & Implementation
Run the cells below before you do anything else. Don't run the colab cells if you're not working in colab!

In [ ]:
### GOOGLE COLAB SETUP -- ONLY RUN IF WORKING IN COLAB ###
! git clone https://github.com/SamBT/manifold-capacity-tutorial-iaifi25 
%cd /content/manifold-capacity-tutorial-iaifi25/
! pip install -r requirements.txt

In [ ]:
### GOOGLE COLAB SETUP -- RUN THIS CELL AFTER THE KERNEL RESTARTS ###
%cd /content/manifold-capacity-tutorial-iaifi25/

In [ ]:
# Imports
#### RUN THIS CELL FIRST ####

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
cvxopt.solvers.options['show_progress'] = False

## 1.1 - Lecture recap

In lecture, we discussed the concept of a **neural manifold**: the ensemble of neural activations linked to the perception of a specific kind of object, e.g. a cat or a dog. The notion of "invariant object recognition" - our ability to quickly recognize a cat or a dog in many different contexts ("views") -- suggests that ours brain processes visual signals into neural manifolds that are well-separated in neural activation space, requiring little additional processing for us to "classify" them into a known category. This idea is illustrated in the figure below ([source](https://www.nature.com/articles/s41467-020-14578-5))

<p align='center'>
    <img src="images/neural_manifold.png" style="width:50%">
</p>

In mathematical language, we characterize such an arrangement as **linearly separable** - i.e. I can draw a hyperplane in neural activation space such that each manifold lies fully on one side or the other. In more colloquial language, we might describe the manifolds as "untangled".

Given this mathematical picture, a number of questions naturally come to mind:

1. How many manifolds can be "packed" into neural activation spaces such that they remain linearly separable?
2. How does this packing relate to the intrinsic dimension or extent of the neural manifolds?
3. How can we mathematically analyze the linear separability of *manifolds* as opposed to *points*, which is the usual setting for linear classification problems (e.g. [perceptrons](https://en.wikipedia.org/wiki/Perceptron) or [support vector machines](https://en.wikipedia.org/wiki/Support_vector_machine))?

These are the some of questions that [manifold capacity theory](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.031003) was developed to answer. 

Manifold capacity turns out to be useful beyond the scope of neuroscience, most notably in AI. Neural networks trained to distinguish cats from dogs are doing essentially the same thing we describe above: transforming raw inputs into some kind of latent representation (i.e. the final hidden layer of a network) where object classes are linearly separable. Neural networks also provide a useful sandbox to investigate manifold capacity theory.

In this tutorial, we'll be implementing functions to compute the mean-field manifold capacity, radius, and dimension discussed in lecture and presented in detail in the [original paper]((https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.031003)). Tutorials 2 and 3 use these tools to analyze representations in neural networks.

## 1.2 - Mean field theory of manifold capacity

The original paper lays out a detailed and mathematically rigorous theory of manifold capacity using statistical mean field theory, the details of which are well beyond the scope of this tutorial. We will take the mean-field results for granted and work on implementing the calculations in python.

The basic model considers a collection of $P$ manifolds $\{M_\mu\}_{\mu=1}^P$ embedded in $\mathbb{R}^N$, each of which consists of a compact subset of a $D$-dimensional affine subspace of $\mathbb{R}^N$, with $D \ll N$. Each is represented by a point cloud $M^\mu = \{\mathbf{x}^\mu_i\}_{i=1}^{n_\mu}$, and we parametrize points on the manifold in terms of an orthonormal basis for the subspace containing $M^\mu$:

$$\mathbf{x}^\mu = \sum_{i=1}^{D+1}s_i^\mu\mathbf{u}_i = \mathbf{x}_0^\mu + \sum_{i=1}^{D}s_i^\mu\mathbf{u}_i$$

This setup is visualized in the picture below:

<p align='center'>
    <img src='images/manifold_model.png' width='50%'>
</p>

We use the notation $\vec{S} = (S_1,S_2,\ldots,S_{D+1}) \in \mathbb{R}^{D+1}$ to denote a point on a manifold in its $D+1$-dimensional basis, and $\mathcal{S} = \{\vec{S}_1,\vec{S}_2,\ldots,\vec{S}_n\}$ for the set of points associated to a manifold. Note that we need $D+1$ coordinates to describe $D$-dimensional subspace because it is not generally centered at the origin.

We're interested in computing the "manifold capacity" $\alpha$, which roughly translates to the maximal number of manifolds per dimension that are linearly separable in $\mathbb{R}^N$ ($P/N$). This turns out to be a pretty complex problem in general, but analyzing it in the thermodynamic limit $P,\,N \to \infty$ with a fixed load $\alpha = P/N$ makes the problem tractable. This analysis yields a mean field theory for computing three quantities for a point cloud representing a manifold in $\mathbb{R}^N$:

1. **Manifold Capacity** $\alpha_M$: the maximum number of such manifolds per dimension ($P/N$) that can be linearly separated in $\mathbb{R}^N$.
2. **Manifold Dimension** $D_M$: The intrinsic dimension of the manifold
3. **Manifold Radius** $R_M$: The "size" or extent of the manifold in the ambient space.

That's about as much as we need to know, but I encourage you to read the paper for more details! **Side note**: for visualization/conceptual purposes, it's easier to think of a manifold as the convex hull of its point cloud. These are the objects we're trying to separate.

## 1.3 - Preprocessing

Our collection of manifolds comes to us as a set of point clouds $\{X_1, \ldots, X_n\}$ where $X_k \in \mathbb{R}^{N \times P_k}$ ($P_k =$ number of points in manifold $k$). The first thing we'll implement is some basic **preprocessing** of each point cloud, which will make the capacity calculations a little easier. This consists of two steps:
1. Center all points across all manifolds about the global mean of all the points in $\mathbb{R}^N$: just like you would center your data before training a neural network.
2. For each manifold, compute a $D+1$-dimensional basis for the affine subspace it lives in, and express points on the manifold in this basis (i.e. the $\vec{S}$ notation used above)

For the manifold bases, we will follow the convention of the original capacity paper and set the $D+1$-th basis vector to be the manifold's center $\mathbf{c}$ (i.e. the mean of the point cloud for the manifold). We'll then express the remaining $1,\ldots,D$ orthonormal basis vectors relative to this center. The coordinates for the center $\mathbf{c}$ will be $\vec{C} = (0,0,\ldots,0,1)$, while the coordinates for any point $\vec{S}$ will be $\vec{S} = (S_1,S_2,\ldots,S_D,1)$.

Let's start with normalization. We want to do two things:
1. **Global centering**: described above; shift all points across all manifolds such that their global mean is zero.

2. **Local centering/normalization for each manifold**: to make downstream computations easier, we'll use "local" coordinates for each manifold. We want to independently shift each point cloud to have zero mean, and scale each point by the norm of its center location. In other words, for each point cloud we do: $$\mathbf{x}_i \mapsto \frac{\mathbf{x}_i - \mathbf{c}}{||\mathbf{c}||}, \quad \mathbf{c} = \frac{1}{P}\sum_{i=1}^{P}\mathbf{x}_i$$

#### **FILL IN CODE**

In [ ]:
def center_manifolds(X):
    """
    Preprocess the set of manifolds by centering all the points about the origin, then center and normalize
    the points of each manifold w.r.t to its center location/norm.
    
    Args:
        X: list[np.ndarray]: list of point clouds, each a numpy array of shape (N, P_i) where N is the
        dimensionality of the ambient space and P_i is the number of points in the i-th manifold.
        
    Returns:
        list[np.ndarray]: Preprocessed manifolds
        list[float]: List of norms of the centers of each manifold.
    """
    # If we have more than one manifold point cloud, 
    # compute the global mean vector across all point clouds
    # and center all the points about this global mean.
    if len(X) > 1:
        # Compute the mean across all manifolds
        X_global_mean = ... # Your code here

        # center all manifolds about the global mean
        X_centered = [...] # Your code here
    else:
        X_centered = X
    
    # Compute centers (means) for each manifold and center/normalize the points of each manifold around its center
    manifold_means = [...] # Your code here
    manifold_mean_norms = [...] # Your code here
    X_normalized = [...] # Your code here
    
    return X_normalized, manifold_mean_norms

Next, we compute orthonormal bases for each centered/normalized manifold. There are two relevant cases:
1. $P_i \geq N$: there are more points than ambient dimensions, in which case the standard basis in $\mathbb{R}^N$ will suffice.

2. $P_i < N$: there are fewer points than ambient dimensions, in which case our point cloud vectors span an at-most $P_i$-dimensional subspace of $\mathbb{R}^N$. We want to compute an orthonormal basis $(\mathbf{e}_1,\ldots,\mathbf{e}_D)$ for this space and express the points in $X_i$ in those coordinates. We can use the [**QR decomposition**](https://en.wikipedia.org/wiki/QR_decomposition), which decomposes a matrix $A \in \mathbb{R}^{m \times n},$ with $m \geq n$ into $A = QR$, where $Q$ is an $m \times n$ matrix with orthonormal columns, and $R$ is an upper triangular $n \times n$ matrix. This is essentially just the [Gram-Schmidt orthonormalization](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) of the columns of $A$, and thankfully `np.linalg.qr` can do it for us.

#### **FILL IN CODE**

In [ ]:
def project_to_manifold_bases(X):
    """
    Compute an orthonormal basis for each manifold using QR decomposition. Express points on each manifold in this basis, and add 
    an additional dimension for the center of the manifold. Each point should have coordinates of the form (S_1, S_2, ..., S_D, 1) 
    where D is the dimension of the manifold, and the last coordinate is always 1 (for the center).

    Args:
        X: list[np.ndarray]: list of point clouds, each a numpy array of shape (N, P_i). These should be the preprocessed manifold data.

    Returns:
        list[np.ndarray]: list of point clouds in their new orthonormal bases.
    """
    X_new_basis = []
    for x in X:
        # Get dimensions/number of points in ambient space
        N, P_i = x.shape
        if N > P_i:
            # Perform QR decomposition
            Q, R = np.linalg.qr(x,mode='reduced') # Q has shape 
            # Project the points onto the orthonormal basis
            x = ... # Your code here
        
        # add the center coordinate (1) as an aditional dimension for each point
        N, new_dim = x.shape
        x = np.concatenate((x, np.ones((1, new_dim))), axis=0)
        # record the coordinates in the new basis
        X_new_basis.append(x)
    return X_new_basis

## 1.4 - How to calculate $\alpha_M$

Now we're ready to do some capacity calculations! Capacity generally depends on the desired sepration margin $\kappa$ between manifolds, though we usually only consider the zero-margin setting $\kappa = 0$ (i.e. a random dichotomy of the manifolds is "just barely separable"). Despite this, we will implement our code for the general case $\kappa \geq 0$.

The key result from mean field theory is the following expression for the manifold capacity:

$$\alpha_M^{-1} = \langle F(\vec{T}) \rangle_{\vec{T}}$$

where the average is performed over random $D+1$-dimensional vectors $\vec{T} \sim \mathcal{N}(\mathbf{0}_{D+1},\mathbf{I}_{D+1})$, and

$$\begin{equation}F(\vec{T}) = \min_{\vec{V}}\{ ||\vec{V} - \vec{T}||^2 \, | \, \vec{V}\cdot\vec{S} - \kappa \geq 0, \; \forall \vec{S} \in \mathcal{S}\}.\end{equation}$$

Recall that $\mathcal{S}$ simply denotes the points on the manifold in terms of their $D+1$-dimensional coordinates.

This seems a bit tricky to solve, but thankfully there useful tools. If we squint our eyes a bit we can immediately recognize that our problem resembles the [standard form](https://en.wikipedia.org/wiki/Optimization_problem#Continuous_optimization_problem) for constrained optimization:

$$
\begin{align*}
\text{min}&\text{imize } f(\mathbf{x}) \\
\text{sub}&\text{ject to } \\
&g_i(\mathbf{x}) \leq 0 \\
&h_j(\mathbf{x}) = 0
\end{align*}
$$

where $i=1,\ldots,m$ and $j=1,\ldots,\ell$ enumerate the constraint functions. In our case, we have $|\mathcal{S}|$ constraints for each $\vec{S}_i$ in the manifold:

$$
\begin{align*}
f(\vec{V}) &= ||\vec{V} - \vec{T}||^2 \\
g_i(\vec{V}) &= -\vec{S}_i\cdot\vec{V} + \kappa, \; i=1,\ldots,|\mathcal{S}|
\end{align*}
$$

Before we get into solving this, we note there are two distinct cases, one of which leads to an easy solution:

1. $\vec{T} \cdot \vec{S} - \kappa \geq 0 \; \forall \vec{S} \in \mathcal{S}$ -- in this case $\vec{V} = \vec{T}$ is a valid and optimal solution! We can check for this case by checking $\min\{\vec{T}\cdot\vec{S} \,| \, \vec{S} \in \mathcal{S}\} - \kappa \geq 0$.
2. The opposite of the above, i.e. $\vec{T} \cdot \vec{S} - \kappa < 0$ for some $\vec{S}$. In this case, we'll actually need to solve the optimization problem.

### The KKT conditions for constrained optimization
As discussed in the [capacity paper](https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.031003), the [Karush-Kuhn-Tucker (KKT) conditions](https://en.wikipedia.org/wiki/Karush%E2%80%93Kuhn%E2%80%93Tucker_conditions) are a helpful way to characterize the solution $\vec{V}^\ast$ to the optimization problem. We're interested in finding a saddle point of the following [Lagrangian function](https://en.wikipedia.org/wiki/Lagrange_multiplier):

$$
\mathcal{L}(\vec{V},\boldsymbol{\lambda}) = f(\mathbf{x}) + \sum_{i=1}^{|\mathcal{S}|}\lambda_i g_i(\mathbf{x}).
$$

The KKT conditions state that the solution $(\vec{V}^\ast,\boldsymbol{\lambda}^\ast)$ satisfies the following conditions:

$$
\begin{align*}
&\nabla f(\vec{V}^\ast) + \sum_{i=1}^{|\mathcal{S}|}\lambda_i^\ast \nabla g_i(\vec{V}^\ast) = 0 \\
&g_i(\vec{V}^\ast) \leq 0 \\
&\lambda_i^\ast \geq 0 \\
&\lambda_i^\ast g_i(\vec{V}^\ast) = 0
\end{align*}
$$
where last three hold for all $i=1,\ldots,|\mathcal{S}|$. To make the analysis simpler, we consider minimizing $\tilde{f}(\vec{V}) = \frac{1}{2}||\vec{V}||^2 - \vec{V}\cdot\vec{T}$ instead of $f(\vec{V}) = ||\vec{V}-\vec{T}||^2$, because they both have the same optimum (**exercise**: check this). The first KKT condition then reads:

$$\vec{V}^\ast - \vec{T} - \sum_{i=1}^{|\mathcal{S}|}\lambda_i^\ast \vec{S}_i = 0,$$

which we can rearrange as

$$\vec{V}^\ast = \vec{T} + \sum_{i=1}^{|\mathcal{S}|}\lambda_i^\ast \vec{S}_i.$$

If we now define $\Lambda = \sum \lambda_i^\ast$ and $\tilde{\lambda}_i^\ast = \lambda_i^\ast/\Lambda$, we can write:

$$ \vec{V}^\ast = \vec{T} + \Lambda \sum_{i=1}^{|\mathcal{S}|}\tilde{\lambda}_i^\ast \vec{S}_i = \vec{V}^\ast + \Lambda \tilde{S}(\vec{T})$$

Why have we done this? If you look closely, you'll notice $\tilde{S}(\vec{T})$ is on the **convex hull** of our manifold. We call this an **anchor point** of the manifold for the sample $\vec{T}$. What the KKT conditions tell us is that we can decompose our solution $\vec{V}^\ast$ into $\vec{T}$ and an anchor point on the convex hull scaled by $\Lambda$. These anchor points turn out to be exactly what we need to compute the manifold capacity, radius, and dimension.

First, let's make a few useful observations:
1. Our decomposition of $\vec{V}^\ast$ allows us to write $$ F(\vec{T}) = ||\Lambda\tilde{S}(\vec{T})||^2, $$ directly tying the calculation of manifold capacity $\alpha_M^{-1} = \langle F(\vec{T}) \rangle$ to the calculation of these anchor points.

2. We wrote our optimization problem with $|\mathcal{S}|$ constraints, but we could have written it with just **one** constraint defined by the **support function** $$g_\mathcal{S}(\vec{V}) = \min_{\vec{S} \in \mathcal{S}} \vec{V}\cdot \vec{S} \implies \text{constraint: } g_\mathcal{S}(\vec{V}) - \kappa \geq 0.$$ This looks simpler but sadly support functions are not differentiable everywhere, so we need to replace the gradients in the first KKT condition with [subgradients](https://en.wikipedia.org/wiki/Subderivative) (which are equal to the usual gradient when $g_\mathcal{S}$ is differentiable). Under this variant of the problem, the KKT conditions tell us: $$ \vec{V}^\ast = \vec{T} + \mu \partial g_S(\vec{V}^\ast),$$ where $\partial$ denotes a subgradient and $\mu$ is a *single* Lagrange multiplier. Comparing this to what we've just done, we can identify $\mu = \Lambda = \sum\lambda_i^\ast$ and $\tilde{S}(\vec{T}) = \partial g_\mathcal{S}(\vec{V}^\ast)$ -- in other words, the anchor point is the unique subgradient of $g_\mathcal{S}$ satisfying the KKT conditions! **When $g_\mathcal{S}$ is differentiable**, we can thus identify: $$ \tilde{S}(\vec{T}) = \argmin_{\vec{S} \in \mathcal{S}} \vec{V}^\ast \cdot \vec{S}$$

3. If we are in the "easy" case $\min\{\vec{T}\cdot\vec{S} \,| \, \vec{S} \in \mathcal{S}\} - \kappa \geq 0$ where we immediately see $\vec{V} = \vec{T}$ is the solution and thus don't need to solve the optimization problem, we can still identify the anchor point as $\tilde{S}(\vec{T}) = \partial g_\mathcal{S}(T) = \argmin_{\vec{S}\in\mathcal{S}}\vec{T}\cdot\vec{S}$

4. If we are not in the trivial case, we can solve the optimization problem for $(\vec{V}^\ast,\boldsymbol{\lambda}^\ast)$ and compute the anchor vector as: $$\tilde{S}(\vec{T}) = \frac{\vec{V}^\ast - \vec{T}}{\Lambda}$$

5. Outside the trivial case, there exists a self-consistent equation for $\Lambda$ ($\mu$) (the details are beyond the scope of this tutorial): $$ \Lambda = \frac{\max(-\vec{T}\cdot \tilde{S}(\vec{T}) + \kappa, 0)}{||\tilde{S}(\vec{T})||^2} $$

### Computing Capacity, Radius, and Dimension
These anchor vectors turn out to be very useful: they are the ingredients we need for computing manifold capacity, radius, and dimension. They are defined as follows:

1. **Capacity**: $\alpha_M^{-1} = \langle ||\Lambda \tilde{S}(\vec{T})||^2 \rangle_{\vec{T}}$

2. **Radius**: $R_M^2 = \langle ||\tilde{s}||^2 \rangle_{\vec{T}}$

3. **Dimension**: $D_M = \langle (\vec{t}\cdot \hat{s})^2 \rangle_{\vec{T}}$

where $\tilde{s}$ and $\vec{t}$ refer to vectors of the first $D$ coordinates of $\tilde{S}$ and $\vec{T}$ excluding the $D+1$-th coordinate that specifies the center, and $\hat{s}$ is a unit vector in the direction of $\tilde{s}$. The computation of capacity uses the full vector $\tilde{S}$.

$R_M$ and $D_M$ both have nice intuitive interpretations related to the manifold's geometric structure. $R_M$ is simply the mean squared norm of the anchor points -- i.e. points on the manifold or its convex hull -- and naturally measures the manifold's spatial extent. $D_M$ essentially measures the average cosine between a random direction $\vec{T}$ in $\mathbb{R}^D$ and the corresponding anchor point on the manifold. If most of the time there is an anchor point roughly along the same direction as a randomly sampled $\vec{t}$, the manifold "fills all $D$ dimensions" and $D_M \approx \langle ||\vec{t}||^2 \rangle = D$. Otherwise, $D_M$ will generically be lower than $D$ depending on the manifold's intrinsic dimensionality (note that $D_M$ is upper bounded by $D$).

## 1.5 - Finally, some code again!

We might be in danger of losing the forest for the trees here, but don't worry -- now we have all the tools we need! Earlier we wrote functions to compute the centered/normalized representation of a manifold $X_k \in \mathbb{R}^{D+1 \times P_i}$ in its local $D+1$-dimensional coordinate system. Starting with this, we compute capacity/radius/dimension calculations in three steps:

1. Randomly sample a bunch of $\vec{T} \in \mathcal{N}(\mathbf{0}_{D+1},\mathbf{I}_{D+1})$
2. For each $\vec{T}$, compute the anchor point $\tilde{S}(\vec{T})$ on the manifold (this is the constrained minimization problem we talked at length about)
3. Compute $\alpha_M$, $R_M$, $D_M$ as above.

Step 1 is easy, so let's get to work coding up step 2!

### Computing anchor points

Let's implement the following algorithm:

**Given**: Manifold point cloud $X = (\vec{S}_1,\ldots,\vec{S}_P) \in \mathbb{R}^{D+1 \times P}$ and a randomly sampled $\vec{T} \sim \mathcal{N}(\mathbf{0}_{D+1},\mathbf{I}_{D+1})$

**Goal**: Determine the minimizer $\vec{V}^\star$ for the function $f(\vec{V}) = ||\vec{V} - \vec{T}||^2$ with the constraint $X^T\vec{V} - \kappa \geq \vec{0}$

**Steps**:

* If $\min\{\vec{T} \cdot \vec{S} - \kappa \; | \; \vec{S} \in \mathcal{S}\} \geq 0$:
    * $\vec{V} = \vec{T}$ is the minimizer and satisfies the constraint.
    * $\tilde{S}(\vec{T}) = \argmin_{\vec{S} \in \mathcal{S}}\vec{V}\cdot \vec{S}$
* Otherwise:
    * Use `cvxopt` to find the minimizer $\vec{V}^\ast$ and the associated Lagrange multipliers $\lambda^\ast_i$ for $i=1,\ldots,P$
    * Compute $\tilde{S}(\vec{T}) = (\vec{V}^\ast - \vec{T})/\sum_{i} \lambda_i^\ast$
    * Compute $\Lambda = \max(-\vec{T}\cdot\tilde{S}(\vec{T}) + \kappa,0)/||\tilde{S}(\vec{T})||^2$

In practice, we will use `numpy` vectorized operations to do this for a whole batch of sampled $\vec{T}$ at once.

#### **FILL IN CODE**

In [ ]:
def samples_and_anchor_points(S, kappa, num_t_samples=300):
    """
    Prepare for the capacity/dimension/radius calculations by sampling num_t_samples random vectors T and computing the anchor points for each
    
    Args:
        S: np.ndarray: shape (D+1,P_i) -- The point cloud representing the manifold in its centered/normalized basis coordinates
                                          S = (S_1, S_2, ..., S_P) where S_i is a D+1-dim vector.
        
        kappa: float: the manifold capacity separation margin.
        
        num_t_samples: int: the number of random vectors T to sample; 300 is a reasonable default

    Returns:
        T_samples: np.ndarray: shape (D+1, num_t_samples) -- the randomly sampled vectors T in R^(D+1)
        S_tildes: np.ndarray: shape (D+1, num_t_samples) -- The anchor points for each sampled vector in T_samples
    """
    
    D1, m = S.shape # D1 = D+1, m = number of points in the manifold
    D = D1 - 1 # D is the dimension of the manifold without its center coordinate
    
    # Sample num_t_samples random vectors T of dimension D1 from a standard normal distribution
    T_samples = ... # Your code here, array of shape (D1, num_t_samples)

    # compute the anchor points for each T sample
    S_tildes = compute_anchor_points(S,T_samples,kappa) # implement this function below

    return T_samples, S_tildes

def compute_anchor_points(S, T_samples, kappa, eps=1e-8):
    """
    For each sampled vector in T, compute the anchor point S_tilde. This involves solving min_V (||V - T||^2 | V.s - kappa >= 0 for all s in S) for each t in T.
    
    Args:
        S: np.ndarray: (D+1, P_i) -- The point cloud representing the manifold in its basis coordinates, shape (D+1,P_i).
        T_samples: np.ndarray: (D+1, N_T) -- a collection of num_t_samples vectors in R^D+1, sampled from a standard normal distribution.
        kappa: the manifold capacity separation margin.

    Returns:
        S_tildes: np.ndarray: (D+1, num_t_samples) -- The anchor points for each sampled vector in T_samples
    """

    num_t_samples = T_samples.shape[1]  # number of random T samples

    # For each T, compute its minimum projection onto the points in S -- this will help us determine if there is an easy solution V = T
    projections, S_min_projections = min_projection(T_samples, S) # implement this function below

    # for each random T, compute the anchor point s_tilde and collect them together
    S_tildes = [] 
    for i in range(num_t_samples):
        T = T_samples[:, i]
        
        # check if there is a V = T solution, by checking the min projection
        if projections[i] - kappa >= 0:
            Vstar = ... # Your code here
            s_tilde = ... # Your code here
        else:
            # compute the optimal V using a solver routine (below)
            Vstar, lambdas, min_VT2 = minimize_VminusT2(T, S, kappa) # go fill in this function (most of it is done for you)

            if np.linalg.norm(Vstar - T) < eps: 
                # if the solution is really close to T, it's probably the "easy" case in reality and numerical oddities happened
                # this should be exactly what you did in the first if case above
                Vstar = ...
                s_tilde = ...
            else:
                # use Vstar and the Lagrange multiplier alpha to compute the anchor point s_tilde
                Lambda = np.sum(lambdas) # the sum of the Lagrange multipliers
                s_tilde = ... # Your code here
        
        S_tildes.append(s_tilde.reshape(-1,1)) # reshape to (D+1, 1) for each anchor point
    
    # concat S_tildes to be a (D+1, num_t_samples) array
    S_tildes = np.concatenate(S_tildes,axis=1)

    return S_tildes


def min_projection(T, S):
    """
    Compute the minimum projection of each vector of T onto the points in S.
    
    Args:
        T: np.ndarray: (D+1,N_T) -- a collection of N_T vectors in R^D+1
        S: np.ndarray: (D+1,P_i) -- the point cloud representing the manifold in its basis coordinates

    Returns:
        min_projections: np.ndarray: (N_T,) -- the minimum projection of each vector in T onto the points in S
        S_min_projections: np.ndarray: (D+1,N_T) -- the points in S which correspond to the minimum projections
    """
    # compute a matrix M_ij = T_i dot S_j, where T_i is the i-th vector in T and S_j is the j-th point in S
    # should  have shape (N_T, P_i)
    M = ... # Your code here

    # get the minimum projection for each vector in T
    min_projections = ... # Your code here

    # retrieve the vectors in S which correspond to the minimum projections (hint: use argmin)
    S_min_projections = S[...] # Your code here

    return min_projections, S_min_projections

def minimize_VminusT2(T, S, kappa):
    """
    Minimize the function ||V - T||^2 subject to the constraint V.s - kappa >= 0 for all s in S. We use the cvxopt library to solve this constrained optimization problem.
    As discussed in the theory section, we actually minimize 0.5V^2 - V.T

    This function is mostly filled in for you. You just need to prepare the matrices for cvxopt 
    
    Here's the prolem setup:
        cvxopt wants the problem in the form of a quadratic program:
            minimize 0.5 x^T P x + q^T x
            subject to Gx <= h
            where P and G are matrices
        
        In our case x = T and we have:
            f(x) = 0.5 ||x||^2 - x . T
            subject to V.s - kappa >= 0 for all s in S (i.e. s = columns of S which is of shape (D+1, P))
            
            P = I (identity matrix)
            q = -T
        
        We write our constraint as:
            matmul(S_transpose, V) - kappa >= 0 (kappa = a vector of shape (P,) with all entries equal to kappa)
            --> matmul(-S_transpose, V) <= -kappa
            which means:
                G = -S_transpose 
                h = -kappa
        
    
    Args:
        T: np.ndarray: (D+1,) -- a vector in R^D+1
        S: np.ndarray: (D+1,P_i) -- the point cloud representing the manifold in its basis coordinates
        kappa: float -- the separation margin

    Returns:
        Vstar: np.ndarray: (D+1,) -- the optimal vector V which minimizes ||V - T||^2 subject to the constraint
        alpha: float -- the Lagrange multiplier associated with the constraint
        min_VT2: float -- the minimum value of the original objective function ||V - T||^2
    """
    D1 = T.shape[0]
    m = S.shape[1] 

    # set up the quadratic program
    ## FILL IN CODE HERE ##
    P = cvxopt.matrix(...)  # identity matrix of shape (D1,D1)
    q = cvxopt.matrix(...) # q = -T
    G = cvxopt.matrix(...) # G = -S_transpose
    h = cvxopt.matrix(...) # h = -kappa (*vector* of shape (m,) not a scalar!)

    # solve the quadratic program
    sol = cvxopt.solvers.qp(P, q, G, h)

    # retrieve the outputs we want: Vstar and the Lagrange multiplier alpha
    Vstar = np.array(sol['x']).flatten() # optimal V
    lambdas = np.array(sol['z']).flatten() # Lagrange multipliers
    min_VT2 = np.sum((Vstar-T)**2) # the minimum value of the original objective function ||V-T||^2

    return Vstar, lambdas, min_VT2

With samples and anchor points in hand, let's implement routines to compute $\alpha_M$, $R_M$ and $D_M$.

In [ ]:
def compute_capacity(T_samples, S_tildes, kappa):
    """
    Compute the manifold capacity alpha_M = 1 / <F(T)>, where F(T) = ||Lambda * S_tilde(T)||^2 

    Args:
        T_samples: np.ndarray: (D+1, num_t_samples) -- a collection of num_t_samples vectors in R^D+1, sampled from a standard normal distribution.
        S_tildes: np.ndarray: (D+1, num_t_samples) -- the anchor points for each sampled vector in T_samples
        kappa: float -- the manifold capacity separation margin
    """
    # Computing Lambda = max(-T . S_tilde + kappa, 0) / ||S_tilde||^2

    # first compute a vector of ||S_tilde||^2 for each s_tilde (for the denominator); shape (num_t_samples,)
    S_tilde_normSquared = ... # Your code here

    # now compute the Lambda values
    lam = ... # Your code here; shape (num_t_samples,)

    # compute F(T) = (Lambda^2) * ||S_tilde||^2
    FT = ... # Your code here; F(T) for each T; shape (num_t_samples,)

    # compute the manifold capacity alpha_M = 1 / <F(T)>
    alpha_M = ...

    return alpha_M

def compute_radius(S_tildes):
    """
    Compute the radius of the manifold, which is the average norm of the first D coordinates of the anchor points S_tildes.
    
    Args:
        S_tildes: np.ndarray: (D+1, num_t_samples) -- the anchor points for each sampled vector in T_samples

    Returns:
        radius: float -- the radius of the manifold
    """
    D1, nsamples = S_tildes.shape  # D1 = D + 1
    D = D1 - 1

    # account for any nonzero mean we might have among the anchor points, even though the manifold mean overall should be zero
    mean_S = np.mean(S_tildes,axis=1,keepdims=True)
    center_S = S_tildes[-1, :] # the center coordinate of each anchor point (in theory this is always 1, in practice it might not be, so we scale our vectors by it)
    S_centered_scaled = (S_tildes - mean_S) / center_S
    
    # take just the first D coordinates of each anchor point
    s = ... # your code here

    # compute the R_M = sqrt(<||s||^2>)
    R_M = ... # your code here

    return R_M

def compute_dimension(T_samples, S_tildes):
    """
    Compute the dimension of the manifold, D = <(s . t)^2> 
    
    Args:
        T_samples: np.ndarray: (D+1, num_t_samples) -- a collection of num_t_samples vectors in R^D+1, sampled from a standard normal distribution.
        S_tildes: np.ndarray: (D+1, num_t_samples) -- the anchor points for each sampled vector in T_samples

    Returns:
        dimension: float -- the dimension of the manifold
    """
    D1, nsamples = S_tildes.shape  # D1 = D + 1
    D = D1 - 1

    # grab the first D coordinates of S_tilde and T
    s = ... # Your code here
    t = ... # Your code here

    # compute unit vectors
    s_hat = ... # Your code here
    t_hat = ... # Your code here

    # compute the dimension D_M = D * <(s_hat . t_hat)^2>
    D_M = ...

    return D_M

We finally have functions to compute everything we want! Now let's wrap it all up into a single function

Given a list of manifold point clouds $[X_1,\ldots,X_m]$, our function will compute the capacity, radius, and dimension for each.

In [ ]:
def compute_manifold_quantities(X, kappa, num_t_samples=300, return_anchor_points=False):
    """
    Compute various manifold quantities from the input data.

    Args:
        X: list[np.ndarray] -- manifolds [X_1, ..., X_m], where each X_i is a manifold point cloud with shape (N, P_i) 
        kappa: float -- the manifold capacity separation margin
        num_t_samples: int -- number of randomly sampled vectors to use in capacity calculation

    Returns:
        alpha_M: list[float] -- the manifold capacities computed for each manifold
        R_M: list[float] -- the radii computed for each manifold
        D_M: list[float] -- the dimensions computed for each manifold
    """
    
    # first center/normalize the data
    X_centered_normed, mean_norms = center_manifolds(X)
    
    # express manifold point clouds in their D+1-dimensional bases
    X_proj = project_to_manifold_bases(X_centered_normed)

    # compute capacities, radii, and dimensions for each manifold
    alpha_M = []
    R_M = []
    D_M = []
    anchor_points = []
    for Xp in X_proj:
        T_samples, S_tildes = samples_and_anchor_points(Xp, kappa, num_t_samples)
        
        a = compute_capacity(T_samples, S_tildes, kappa)
        R = compute_radius(S_tildes)
        D = compute_dimension(T_samples, S_tildes)

        alpha_M.append(a)
        R_M.append(R)
        D_M.append(D)
        anchor_points.append(S_tildes)
    alpha_M = np.array(alpha_M)
    R_M = np.array(R_M) * np.array(mean_norms)  # scale the radii by the mean norms of the manifolds
    D_M = np.array(D_M)
    if return_anchor_points:
        return alpha_M, R_M, D_M, anchor_points
    else:
        return alpha_M, R_M, D_M

## 1.6 - Exploring manifold capacity

Now that we've got the tools we need, let's explore how manifold capacity works for a simple class of manifolds: **ellipsoids**. In $\mathbb{R}^N$ these are defined by the equation

$$ \sum_{i=1}^N \frac{x_i^2}{R_i^2} = 1$$

where $R_i$ are the lengths of the various axes of the ellipse. 

We can generate points on an axis-aligned ellipsoid by generating points on the unit sphere, then dilating each dimension the corresponding $R_i$. Complete the code implementation below.

#### **FILL IN CODE**

In [ ]:
def sample_unit_ball(n_samples, n_dim, surface=True):
    """
    Uniformly sample points on the unit sphere or in the unit ball in R^n_dim, default is to sample on the surface (surface=True)

    Args:
        n_samples: number of samples
        n_dim: dimension of the space

    Returns:
        samples: (n_samples, n_dim) array
    """
    # Sample from standard normal shape (n_samples, n_dim)
    x = ... # Your code here
    # normalize each vector to lie on the unit sphere
    x /= ... # Your code here

    if not surface:
    # Scale by r^(1/n) where r ~ Uniform[0, 1]
        r = np.random.rand(n_samples) ** (1 / n_dim)
        x *= r[:, None]

    return x

Now complete the code to generate points on an ellipsoids with axis lengths $[R_1,R_2,\ldots,R_N]$ and add a random offset $\mathbf{c}$ as the center location.

#### **FILL IN CODE**

In [ ]:
def generate_ellipsoid_points(n_points, axes_lengths, radius=None, rcenter=1.0, surface=True):
    """
    Generate points on an N-dimensional ellipsoid with specified axis lengths.
    
    Args:
        n_points (int): Number of points to generate
        axes_lengths (list or numpy.ndarray): The lengths of the semi-axes of the ellipsoid
        radius: float, optional: If specified, scale the axes lengths so that the longest one is equal to this radius
        rcenter: float, optional: radius of the random center offset, defaults to 1
        surface: bool, optional: If True, sample points on the surface of the ellipsoid; 
                                 if False, sample points uniformly in the volume of the ellipsoid.      
        
    Returns:
        numpy.ndarray: Array of shape (n_points, N) containing points on the ellipsoid
    """
    if type(axes_lengths) is not np.ndarray:
        axes_lengths = np.array(axes_lengths)
    
    # Number of dimensions
    N = len(axes_lengths)

    points = sample_unit_ball(n_points, N)  # Generate points on the unit sphere
    
    # scale axes lengths so that the longest one is equal to radius
    if radius is not None:
        axes_lengths = radius * axes_lengths / np.max(axes_lengths)
    
    # Scale each dimension by its axis length to get an ellipsoid
    points = ... # Your code here

    # put the points around a random nonzero center sampled from the unit sphere
    # sample a random center point on the unit sphere
    center = sample_unit_ball(1, N)
    # add the random center offset scaled by rcenter
    points += rcenter * center 

    return points

Finally, let's implement a function that embeds a set of points (ellipsoid or otherwise) in $\mathbb{R}^N$ into a higher-dimensional space $\mathbb{R}^M$ ($M > N$). We'll be sampling relatively low-dimensional ellipsoids, embedding them into a high-dimensional ambient space, then using our manifold capacity tools to analyze their intrinsic dimension/packability. 

#### **FILL IN CODE**

In [ ]:
def embed_to_dim(x,M):
    nsamples,N = x.shape
    assert M > N
    
    # randomly choose N distinct axes in R^M to embed the points in
    rand_coords = ... # Your code here; should have shape N
    
    # initialize the embedding vector as zeros
    embedding = np.zeros((nsamples,M))
    
    # assign the points to the randomly chosen axes
    embedding[...] = x # Your code here
    
    return embedding

Let's do a quick sanity check to make sure our code is working properly:

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

nsamp = 1000
axes_lengths = [2, 10, 5]
rcenter = 1.0
points = generate_ellipsoid_points(nsamp, axes_lengths, rcenter=rcenter)

# Visualize the generated points

# 3D visualization
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:, 0], points[:, 1], points[:, 2], alpha=0.7, s=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Set appropriate axis bounds
center = points.mean(axis=0)
radius = max(axes_lengths)
ax.set_xlim([center[0]-radius, center[0]+radius])
ax.set_ylim([center[1]-radius, center[1]+radius])
ax.set_zlim([center[2]-radius, center[2]+radius])

ax.grid(True)

Now we can put our manifold capacity tools to use. In the cells below we'll generate some ellipsoids of varying dimension, embed them into a fixed high-dimensional space, compute the manifold capacity/dimension, and look at how these quantities reflect the ellipsoid's true dimension. **NOTE**: the manifold capacity code expects to receive a list of point clouds $[p_1,\ldots,p_k]$ with shapes $p_i \in \mathbb{R}^{N\times n_\text{samples}}$, so you'll need to tranpose the point clouds we generate with our functions.

In [ ]:
# feel free to edit these choices!
ambient_dim = 100
ellips_dims = [2,4,6,8,10,15] 
axes_lengths = [d*[1] for d in ellips_dims]  # just do spheres; feel free to change this
nsamples_per = 500
rcenter = 1.0

point_clouds = []
for edim,ax_lengths in zip(ellips_dims,axes_lengths):
    # generate points
    points = generate_ellipsoid_points(nsamples_per, ax_lengths, rcenter=rcenter, radius=1.0)

    # embed in a ambient_dim space
    points = embed_to_dim(points, ambient_dim)

    # add to the point cloud list (careful about the shape!)
    point_clouds.append(points.T)

alphas, Rs, Ds = compute_manifold_quantities(point_clouds, kappa=0, num_t_samples=300)

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(18,5))

plt.sca(axes[0])
plt.plot(ellips_dims, alphas, lw=2)
plt.xlabel('Ellipsoid Dimension')
plt.ylabel('Mean Field Capacity $\\alpha_M$')

plt.sca(axes[1])
plt.plot(ellips_dims, Ds, lw=2)
plt.xlabel('Ellipsoid Dimension')
plt.ylabel('Mean Field Dimension $D_M$')

plt.sca(axes[2])
plt.plot(ellips_dims, Rs, lw=2)
plt.xlabel('Ellipsoid Dimension')
plt.ylabel('Mean Field Radius $R_M$')

#### **Exercises**
Try varying a few parameters and exploring how the mean-field capacity, radius, and dimension change. A few things to consider

1. Does manifold capacity $\alpha_M$ vary in an intuitive way as you change vary the ellipsoid dimension and the ambient dimension?

2. How is the number of samples per manifold vs. the ambient dimension reflected in the "accuracy" of the computed dimension/radius (relative to ground truth)?

3. Are $R_M$/$D_M$ systematically over/underestimated? Under what conditions?